In [1]:
import sys

import csv
import json
import time
import datetime
from ast import literal_eval
import pandas as pd
import numpy as np
#import geohash
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
from pyspark.sql.functions import mean, min, max, sum
from pyspark.sql import functions as func
import glob
import matplotlib.colors
from collections import Counter
import os

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [11]:
datapath=os.environ['LAV_DIR']
figdir=datapath+ "fig/"
resultsdir=datapath+ "out/"
inputdir=datapath+"log/"
fn=inputdir+"jll4h"

In [12]:
time_mapping={2:"2",3:"3",4:"4",5:"5",6:"6",7:"7",8:"8",9:"9",10:"10",11:"11",12:"12",13:"13",14:"14",15:"15",\
             16:"16",17:"17",18:"18",19:"19",20:"20",21:"21",22:"22",23:"23",24:"0+1",25:"1+1"}
hours=[x for x in map(str,range(2,24))]+["0+1","1+1"]

In [13]:
sqlContext = SQLContext(sc)
df = sqlContext.read.parquet(fn)
num_chains=df.count()
print(num_chains)
pf=df.toPandas()

476520


In [18]:
pf.head(4)

,index,act_cell,hour,market_share,first_cilac,count
0,6,2043-5275,19,0.23,26351361-1493,30
1,24,61597-5124,15,0.23,61597-5124,30
2,319,17175-5275,8,0.23,17175-5275,113
3,344,64591-5127,1+1,0.23,25678-5127,30


In [15]:
#this was utc before...
pf["hour"]=pf.apply(lambda row:row.hour+2,axis=1)
pf["hour"]=pf.apply(lambda row:time_mapping[row.hour],axis=1)
pf["count"]=pf.apply(lambda row:int(np.round(row["count"]*(1.0/float(row.market_share)))),axis=1)
pf=pf[pf["count"]>=30]
pf=pf.reset_index()

In [16]:
#make matrix
all_cells=list(np.unique(pf["act_cell"].values))
res=pd.DataFrame(index=all_cells,columns=hours)
for pp in range(0,len(pf)):
    res.loc[pf["act_cell"][pp],pf["hour"][pp]]=pf["count"][pp]
res=res.fillna(0)
res.to_csv(resultsdir+"JLL_activities_test4h.csv")

In [17]:
res.head(4)

,2,3,4,5,6,7,8,9,10,11,...,16,17,18,19,20,21,22,23,0+1,1+1
10052-5127,0,0,91,96,113,91,57,35,39,0,...,39,43,30,0,30,0,48,48,35,39
1095-5275,0,0,39,39,35,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12449-17352,0,0,0,34,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14652-5126,0,0,0,0,43,39,43,0,0,0,...,0,0,0,0,0,0,0,0,0,0
